In [1]:
import torch 

if torch.cuda.is_available():    
 
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [2]:
import os

import numpy as np
import pandas as pd

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset

from sklearn import metrics

from tqdm.notebook import tqdm

import transformers 

import sys; sys.path.insert(0, '..')

In [3]:
test = pd.read_json('../data/raw/test_unseen.jsonl',lines=True)[['id']]
train = pd.read_json('../data/raw/train.jsonl', lines=True)
dev = pd.read_json('../data/raw/dev_unseen.jsonl', lines=True)

In [4]:
train = pd.concat([train,dev],axis=0)[['id','label']]

In [5]:
#FOLDS = 1 # BEST RESULTS
#MODELS = ['f_dbu','f_er','f_rb','f_u','f_u_img_cap'] # BEST RESULTS

FOLDS = 10 # Cross validation
MODELS = ['example_10fold_dbu','example_10fold_er'] # best result with 10 fold of same 5 models above, this is an example

In [6]:
def get_fold_data(f_idx):

    train_fold = pd.DataFrame()
    dev_fold = pd.DataFrame()
    test_fold = pd.DataFrame()

    for fold_idx in range(1,FOLDS+1):

        for m_idx, m in enumerate(MODELS):

            if fold_idx != f_idx:

                train_path = '../artifacts/level-0/'+m+'/preds/dev_'+str(fold_idx)+'.npz'

                tmp = np.load(train_path,allow_pickle=True)
                df = pd.DataFrame(tmp['features'])
                #df = pd.DataFrame()
                df['id'] = pd.Series(tmp['id'])
                #f['probas'] = pd.Series(tmp['probas'])
                #df['pred_labels'] = pd.Series(tmp['pred_labels'])
                if m_idx == 0:
                    f_train_fold = df
                else:
                    f_train_fold = pd.merge(f_train_fold,df,on='id')

            else:
            
                dev_path = '../artifacts/level-0/'+m+'/preds/dev_'+str(f_idx)+'.npz'
                test_path = '../artifacts/level-0/'+m+'/preds/test_'+str(f_idx)+'.npz'

                tmp = np.load(dev_path,allow_pickle=True)
                df = pd.DataFrame(tmp['features'])
                #df = pd.DataFrame()
                df['id'] = pd.Series(tmp['id'])
                #df['probas'] = pd.Series(tmp['probas'])
                #df['pred_labels'] = pd.Series(tmp['pred_labels'])
                if m_idx == 0:
                    dev_fold = df
                else:
                    dev_fold = pd.merge(dev_fold,df,on='id')

                tmp = np.load(test_path,allow_pickle=True)
                df = pd.DataFrame(tmp['features'])
                #df = pd.DataFrame()
                df['id'] = pd.Series(tmp['id'])
                #df['probas'] = pd.Series(tmp['probas'])
                #df['pred_labels'] = pd.Series(tmp['pred_labels'])
                if m_idx == 0:
                    test_fold = df
                else:
                    test_fold = pd.merge(test_fold,df,on='id')
        
        if fold_idx != f_idx:
            train_fold = pd.concat([train_fold,f_train_fold],axis=0)

    if FOLDS == 1:
        train_fold = dev_fold

    train_fold = pd.merge(train_fold,train,on='id')
    dev_fold = pd.merge(dev_fold,train,on='id')
    test_fold = pd.merge(test,test_fold,on='id')

    train_ids = torch.from_numpy(train_fold.id.values)
    train_labels = torch.from_numpy(train_fold.label.values).float()
    train_features = train_fold.drop(['id','label'],axis=1).values
    train_features = torch.stack([torch.from_numpy(row).float() for row in train_features])

    dev_ids = torch.from_numpy(dev_fold.id.values)
    dev_labels = torch.from_numpy(dev_fold.label.values).float()
    dev_features = dev_fold.drop(['id','label'],axis=1).values
    dev_features = torch.stack([torch.from_numpy(row).float() for row in dev_features])

    test_ids = torch.from_numpy(test_fold.id.values)
    test_features = test_fold.drop(['id'],axis=1).values
    test_features = torch.stack([torch.from_numpy(row).float() for row in test_features])

    train_dataset = TensorDataset(train_ids,train_features,train_labels)
    dev_dataset = TensorDataset(dev_ids,dev_features,dev_labels)
    test_dataset = TensorDataset(test_ids,test_features)

    train_dataloader = DataLoader(
        train_dataset,
        sampler=RandomSampler(train_dataset),
        batch_size=16
    )

    dev_dataloader = DataLoader(
        dev_dataset,
        sampler=RandomSampler(dev_dataset),
        batch_size=16
    )

    test_dataloader = DataLoader(
        test_dataset,
        sampler=SequentialSampler(test_dataset),
        batch_size=16
    )

    return train_dataloader, dev_dataloader, test_dataloader


In [7]:
class swish(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x * torch.sigmoid(x)

class mid(nn.Module):

    def __init__(self,hidden_size):
        super().__init__()
        self.intermediate_size = hidden_size
        self.prop = nn.Sequential(
            nn.Linear(self.intermediate_size,self.intermediate_size),
            nn.BatchNorm1d(self.intermediate_size),
            swish(),
            nn.Dropout(0.1)
        )
    
    def forward(self,x):
        return self.prop(x)

class MetaClassifier(nn.Module):

    def __init__(self):
        super().__init__()

        self.hidden_size = 768
        self.intermediate_size = self.hidden_size

        self.input_size = self.hidden_size * len(MODELS) # + 1024 - 768 
        
        self.d1 = nn.Dropout(0.1)
        
        self.input_layer = nn.Sequential(
            nn.Linear(self.input_size,self.intermediate_size),
            nn.BatchNorm1d(self.intermediate_size),
            swish(),
            nn.Dropout(0.1)
        )
        
        layers = nn.ModuleList([mid(self.intermediate_size) for i in range(2)])
        self.prop  = nn.Sequential(*layers)

        #self.cls = nn.Linear(self.intermediate_size,1)
        self.cls = nn.Linear(self.input_size,1)

    def forward(self,inputs):
        """
        x = self.d1(inputs)
        x = self.input_layer(x)
        x = self.prop(x)
        """
        #x = self.cls(x) 
        x = self.cls(inputs)

        return x

In [8]:
def epoch_fn(dataloader, model, optimizer,scaler, device, loss_fn, fp16=False, is_train=True, scheduler = None):
    
    model.train() if is_train else model.eval()

    acumulate_loss = 0

    probas = []
    targets = []
    
    with torch.set_grad_enabled(is_train):
        for bi, batch in tqdm(enumerate(dataloader), total=len(dataloader),leave=False):

            inputs = batch[1].to(device)

            b_targets = batch[2].float().to(device)

            
            with torch.cuda.amp.autocast(enabled=fp16):
                b_logits = model(inputs).float()

            loss = scaler.scale(loss_fn(b_logits, b_targets.view(-1,1)))
            if is_train: 
                loss.backward()
            acumulate_loss += loss.item()
            if not torch.isnan(b_logits).any():
                probas.extend(nn.Sigmoid()(b_logits.squeeze().detach().cpu()).numpy().tolist())
                targets.extend(b_targets.squeeze().detach().cpu().numpy().tolist())

            torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)

            if is_train:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
               #scheduler.step()
    
    print('Loss: ',(acumulate_loss/len(dataloader)))

    auc = metrics.roc_auc_score(targets, probas)
    acc = metrics.accuracy_score(targets, np.array(probas).round())
    
    print(f"Accuracy: {acc}")
    print(f"AUROC: {auc}")

    return auc

def test_probas(model,dataloader,device):

    probas = []

    ids = []

    model.eval()

    with torch.set_grad_enabled(False):
        for bi, batch in tqdm(enumerate(dataloader), total=len(dataloader),leave=False):

            b_ids =  batch[0].detach().numpy()
            inputs = batch[1].to(device)

            with torch.cuda.amp.autocast(enabled=fp16):
                b_logits = model(inputs).float()

            probas.extend(nn.Sigmoid()(b_logits.squeeze().detach().cpu()).numpy().tolist())
            ids.extend(b_ids)

    df = pd.DataFrame()
    df['id'] = ids
    df['probas'] = probas
    return df

In [9]:
from tqdm.notebook import tqdm
from torch.optim.swa_utils import AveragedModel, SWALR
from torch.optim.swa_utils import update_bn
from torch.optim import AdamW, SGD, Adamax 

from transformers import get_linear_schedule_with_warmup
from src.utils.RocStarLoss import RocStarLoss

epochs = 15 # Final train and folds=1, use just 1 or 2 epochs

lr = 2e-5

loss_fn = nn.BCEWithLogitsLoss()
#loss_fn = RocStarLoss(device) # Worse

fp16 = False 
scaler = torch.cuda.amp.GradScaler(enabled=fp16)

In [10]:
model_name = 'example-10-fold-MetaCls'
run_folder = '../artifacts/ensemble/' + model_name + '/'

os.makedirs(run_folder, exist_ok=True)

final_train = False # Then folds must be 1, and there is no validation

In [11]:
test_preds = pd.DataFrame({
    'id':test.id
})

for fold_idx in tqdm(range(1,FOLDS+1),total=FOLDS,leave=False):

    print('------FOLD ',str(fold_idx))

    train_dataloader, dev_dataloader, test_dataloader = get_fold_data(fold_idx)

    model = MetaClassifier().to(device)
    optimizer = AdamW(model.parameters(), lr=lr)

    best_dev_auc = float('-inf')
    valid_auc = float('-inf')

    for epoch in tqdm(range(0,epochs),total=epochs,leave=False):
        print('EPOCH: '+str(epoch))
        print("------- TRAIN -------")
        _ = epoch_fn(train_dataloader, model, optimizer,scaler, device, loss_fn,fp16)
        if not final_train:
            print("------- VALIDATION -------")
            valid_auc = epoch_fn(dev_dataloader, model, optimizer,scaler, device, loss_fn,fp16,is_train=False)
        if (final_train & (epoch == epochs-1)) | (not final_train & (valid_auc > best_dev_auc)):
            if not final_train: best_dev_auc = valid_auc
            print("------- TEST -------")
            f_test_preds = test_probas(model,test_dataloader,device)
            path = run_folder+str(fold_idx)+'.ckpt'
            torch.save(model,path)

    print('\n')
    test_preds = pd.merge(test_preds,f_test_preds,on='id',how='left')

------FOLD  1


EPOCH: 0
------- TRAIN -------


Loss:  0.5184198273071372
Accuracy: 0.7779006882989183
AUROC: 0.8162707042631341
------- VALIDATION -------


Loss:  0.5418486819978345
Accuracy: 0.7809734513274337
AUROC: 0.8229215092202219
------- TEST -------


EPOCH: 1
------- TRAIN -------


...

Loss:  0.4339361502871298
Accuracy: 0.801007866273353
AUROC: 0.8659196384515038
------- VALIDATION -------


Loss:  0.4618935595478928
Accuracy: 0.797566371681416
AUROC: 0.8635461356214382
------- TEST -------


EPOCH: 14
------- TRAIN -------


Loss:  0.4341231540050394
Accuracy: 0.8017453294001966
AUROC: 0.8663290634492125
------- VALIDATION -------


Loss:  0.463384059960382
Accuracy: 0.8008849557522124
AUROC: 0.8632419798937055
------- TEST -------


In [12]:
import gc 
gc.collect()
torch.cuda.empty_cache()

In [13]:
test_preds.shape

(2000, 11)

In [14]:
test_preds['f_proba'] = np.median(test_preds.drop(['id'],axis=1).values,axis=1)
test_preds['f_label'] = test_preds.f_proba.round()
test_preds.shape


(2000, 13)

In [15]:
test_preds.head()

,id,probas_x,probas_y,probas_x,probas_y,probas_x,probas_y,probas_x,probas_y,probas_x,probas_y,f_proba,f_label
0,15740,0.253961,0.151414,0.143838,0.279329,0.285810,0.154840,0.212502,0.305538,0.291787,0.170859,0.233232,0.0
1,38794,0.600004,0.556491,0.781053,0.617369,0.603728,0.566887,0.614959,0.642420,0.583225,0.650743,0.609344,1.0
2,60792,0.167348,0.154422,0.128863,0.246194,0.206585,0.093723,0.235312,0.219494,0.218442,0.297356,0.212513,0.0
3,71824,0.166277,0.102384,0.139526,0.250077,0.205300,0.113272,0.256090,0.245495,0.210237,0.186009,0.195654,0.0
4,4796,0.600243,0.618062,0.741129,0.561387,0.312462,0.589004,0.604261,0.352907,0.630390,0.645550,0.602252,1.0


In [16]:
col_names = ['id','proba','label']

subm = pd.DataFrame(columns=col_names)
subm.id = test_preds.id
subm.label = test_preds.f_label.astype(np.int64)
subm.proba = test_preds.f_proba.astype(np.float64)

print(subm.shape)
print(subm.head())

file_path = '../submissions/'+model_name+'.csv'
subm.to_csv(file_path,index = False, header=True,sep=',',encoding='utf-8-sig')

(2000, 3)
      id     proba  label
0  15740  0.233232      0
1  38794  0.609344      1
2  60792  0.212513      0
3  71824  0.195654      0
4   4796  0.602252      1
